In [1]:
import os
try:
    from google.colab import drive
    COLAB = True
    print("Estoy en Google CoLab")
    %tensorflow_version 2.x
    !pip install livelossplot
except:
    os.environ["CUDA_VISIBLE_DEVICES"]="0" 
    import tensorflow as tf
    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
    print("No estoy en Google CoLab")
    COLAB = False

Estoy en Google CoLab
     |████████████████████████████████| 793 kB 11.1 MB/s 
     |████████████████████████████████| 381 kB 50.2 MB/s 
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.29 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.33.0 which is incompatible.


In [2]:
import keras
import os
import pathlib
import pandas as pd
import numpy as np 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Bidirectional, LSTM, GRU, Dense, MaxPooling2D, Input, Embedding, Concatenate, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Flatten
from tensorflow.keras.layers import Multiply, GlobalAveragePooling1D, Lambda
from  tensorflow.keras import backend as K
from tensorflow.keras import layers as tflayers
from tensorflow.keras import models as tfmodels
from livelossplot import PlotLossesKerasTF
from tensorflow.keras.utils import to_categorical

# Detector de paráfrasis

En ente ejemplo se mostrará como usar capas de embeddings y embeddings pre-entrenados para entrenar un modelo que identifique cuando dos frases son paráfrasis o no. 

Vamos a empezar descargando el conjunto de datos con la función `get_file` de keras que además de descargar el archivo lo puede descromprimir:

In [3]:
data_path = keras.utils.get_file(
    "parafrasis.zip",
    "https://hdvirtual.us.es/discovirt/index.php/s/NZqkjMbTeEnp6w2/download",
    extract=True,
    archive_format='zip'
)

AttributeError: ignored

Mostramos los archivos que contiene:

In [ ]:
os.listdir(pathlib.Path(data_path).parent / 'parafrasis') 

Tenemos un archivo de train y uno de validación (dev) y otro de test. Vamos a hacer la lectura de los archivos (para nuetra prueba vamos a usar train y test):

In [ ]:
train = pd.read_csv(pathlib.Path(data_path).parent / 'parafrasis' / 'train.tsv', sep='\t')
test = pd.read_csv(pathlib.Path(data_path).parent / 'parafrasis' / 'test_2k.tsv', sep='\t')

In [ ]:
train.head()

In [ ]:
train.iloc[1].sentence1, train.iloc[1].sentence2

In [ ]:
test.head()

In [ ]:
train.shape, test.shape

Balanceo de la respuesta:

In [ ]:
1- test.label.mean()

#### Crear un indice del vocabulario

En primer lugar necesitamos asociar a cada palabra (mejor dicho token) un número. Para ellos usaremos la clase `Tokenizer` de Keras:

In [ ]:
corpus = list(train.sentence1.values) + list(train.sentence2.values) + \
         list(test.sentence1.values) + list(test.sentence2.values)
corpus = [str(s) for s in corpus]      
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(corpus)

Una vez *entrenado*, el tokenizador habrá asociado a cada palabra un número o índice:

In [ ]:
tokenizer.word_index

Al vocabulario agregamos un caracter adicional para rellenar las frases que sean más cortas de lo establecido:

In [ ]:
total_tokens = len(tokenizer.word_index) + 1
total_tokens

Convertimos las sentencias a sencuencias de índices y calculamos el tamaño máximo:

In [ ]:
train['sent1_enc'] = tokenizer.texts_to_sequences(train.sentence1.astype('str'))
train['sent2_enc'] = tokenizer.texts_to_sequences(train.sentence2.astype('str'))
test['sent1_enc'] = tokenizer.texts_to_sequences(test.sentence1.astype('str'))
test['sent2_enc'] = tokenizer.texts_to_sequences(test.sentence2.astype('str'))

In [ ]:
max_size = max([train.sent1_enc.map(lambda x: len(x)).max(), train.sent2_enc.map(lambda x: len(x)).max(),
               test.sent1_enc.map(lambda x: len(x)).max(), test.sent2_enc.map(lambda x: len(x)).max()])
max_size

Ahora podemos igualar los tamaños de las sentencias agregar un token de relleno usando `pad_sequences`:

In [ ]:
train['sent1_enc'] = list(pad_sequences(train.sent1_enc, maxlen=max_size, padding='post'))
train['sent2_enc'] = list(pad_sequences(train.sent2_enc, maxlen=max_size, padding='post'))
test['sent1_enc'] = list(pad_sequences(test.sent1_enc, maxlen=max_size, padding='post'))
test['sent2_enc'] = list(pad_sequences(test.sent2_enc, maxlen=max_size, padding='post'))

In [ ]:
train.head()

Creamos los conjuntos de entrenamiento

In [ ]:
X1, X2 = np.array(train.sent1_enc.to_list()), np.array(train.sent2_enc.to_list())
X_val1, X_val2 = np.array(test.sent1_enc.to_list()), np.array(test.sent2_enc.to_list())

## Usando embeddings pre-entrenados

### Lectura de los embeddings

En primer lugar debemos descargar los embeddings pre-entrenados. En este caso vamos a usar GloVe. El arhivo comprimido son (906MB)

In [ ]:
data_path = keras.utils.get_file(
    "glove.zip",
    "https://hdvirtual.us.es/discovirt/index.php/s/dBfebSHNqKfeZTR/download",
    extract=True,
    archive_format='zip'
)

Una vez descargado hacemos la lectura de los embeddings creando un diccionario cuya clave es la palabra y el valor es el vector:

In [ ]:
glove_path = pathlib.Path(data_path).parent / 'SBW-vectors-300-min5.txt'

embeddings_index = {}
with open(glove_path) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Encontrados %s vectors." % len(embeddings_index))

Ahora tenemos que preparar una matriz que usará la capa de embedding de Keras. Esta matriz es solo una matriz de numpy donde la entra i-ésima corresponde al vector de la palabra i-ésima de nuestro vocabulario:

In [ ]:
embedding_matrix = np.zeros((total_tokens, 300))
count = 0
palabras_no_encontradas = []
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        palabras_no_encontradas.append(word)
        count += 1
print("No se encontraron vectores para %d palabras (de %d)" % (count, total_tokens))        

In [ ]:
palabras_no_encontradas

Dado que ahora cada una de nuestras palabras es un vector, podemos hacer operaciones vectoriales. Por ejemplo ¿que pasaría si a la palabra *rey* le quitamos la carga semántica de *hombre* y le sumamos la carga semántica de *mujer*?

In [ ]:
from scipy import spatial
v = embeddings_index.get('rey') - embeddings_index.get('hombre') + embeddings_index.get('mujer') 
sorted([(w, spatial.distance.cosine(v, embeddings_index.get(w)))
        for w in tokenizer.word_index.keys() 
        if embeddings_index.get(w) is not None ],
       key=lambda x: x[1])


## Modelo recurrente bidireccional con distancia coseno

Vamos a necesitar un par de funciones que calculen la distancia coseno entre dos vectores:

In [ ]:
def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

Ahora nos podemos crear una red que recibirá dos entradas. Cada entrada será codificada con los embedings preentrenados y pasarán por las LSTMs bidireccionales. La salidas de las LSTMs (los estados) los usaremos para calcular la distancia coseno usando la capa *Lambda*. Por último con una capa densa obtenemos la salida:

In [ ]:
s1 = Input(shape=(max_size, ), dtype="int64")
s2 = Input(shape=(max_size, ), dtype="int64")

embedding = Embedding(total_tokens, 300, weights=[embedding_matrix], trainable=False, input_length=max_size)

e1 = embedding(s1)
e2 = embedding(s2)

recurrent = Bidirectional(LSTM(64, return_sequences=True))
x = recurrent(e1)
y = recurrent(e2)
z = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([x, y])
z = Flatten()(z)
preds = Dense(2, activation="softmax")(z)

model = Model([s1, s2], preds)
model.summary()

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([X1, X2], to_categorical(train.label.values),
          epochs=10,
          batch_size=32,
          validation_data=([X_val1, X_val2], to_categorical(test.label.values)),
          callbacks=[PlotLossesKerasTF()])

## LSTM Bidireccional con red de relevancia

Basado en el artículo [Bidirectional Long Short-Term Memory with Gated Relevance Network for Paraphrase Identification](https://link.springer.com/chapter/10.1007/978-3-319-50496-4_49)

In [ ]:
from keras.layers import Layer
from tensorflow.keras import backend as K
class GatedRelevanceNetwork(Layer):
    def __init__(self, output_dim,
            weights_initializer="glorot_uniform",
            bias_initializer="zeros", **kwargs):
        self.output_dim = output_dim
        self.weights_initializer = weights_initializer
        self.bias_initializer = bias_initializer
        super(GatedRelevanceNetwork, self).__init__(**kwargs)

    def build(self, input_shape):
        batch_size, len1, emb_dim = input_shape[0]
        _, len2, _ = input_shape[1]
        
        # Weights initialization
        # Bilinear Tensor Product weights
        self.Wb = self.add_weight(name='weights_btp',
                                  shape=(self.output_dim, emb_dim, emb_dim),
                                  initializer=self.weights_initializer,
                                  trainable=True)

        # Pesos de la red totalmente conectada
        self.Wd = self.add_weight(name='weights_sln',
                                  shape=(2*emb_dim, self.output_dim),
                                  initializer=self.weights_initializer,
                                  trainable=True)

        # Pesos de la puerta (ver el artículo)
        self.Wg = self.add_weight(name='weights_gate',
                                  shape=(2*emb_dim, self.output_dim),
                                  initializer=self.weights_initializer,
                                  trainable=True)

        # Sesgo
        self.bg = self.add_weight(name='bias_gate',
                                  shape=(self.output_dim,),
                                  initializer=self.bias_initializer,
                                  trainable=True)

        # Sesgo general
        self.b = self.add_weight(name='bias',
                                 shape=(self.output_dim,),
                                 initializer=self.bias_initializer,
                                 trainable=True)

        # Pesos del canal (ver el artículo)
        self.u = self.add_weight(name="channel_weights",
                                 shape=(self.output_dim, 1),
                                 initializer=self.weights_initializer,
                                 trainable=True)

        super(GatedRelevanceNetwork, self).build(input_shape)

    def call(self, x):
        e1 = x[0]
        e2 = x[1]

        batch_size = K.shape(e1)[0]
        _, len1, emb_dim = K.int_shape(e1)
        _, len2, _ = K.int_shape(e2)

        # Se repiten las matrices para generar todas las combinaciones
        ne1 = K.reshape(K.repeat_elements(K.expand_dims(e1, axis=2), len2, axis=2),
                       (batch_size, len1*len2, emb_dim))
        ne2 = K.reshape(K.repeat_elements(K.expand_dims(e2, axis=1), len1, axis=1),
                       (batch_size, len1*len2, emb_dim))

        # Se repite la segunda matriz para poder hacer el producto bilineal
        ne2_k = K.repeat_elements(K.expand_dims(ne2, axis=-1), self.output_dim, axis=-1)

        # Producto
        btp = K.sum(ne2_k * K.permute_dimensions(K.dot(ne1, self.Wb), (0,1,3,2)), axis=2)
        btp = K.reshape(btp, (batch_size, len1, len2, self.output_dim))

        # Concatemos para pasar por la red totalmente conectada
        e = K.concatenate([ne1, ne2], axis=-1)

        # Red totalmente conectada
        sln = K.tanh(K.dot(e, self.Wd))
        sln = K.reshape(sln, (batch_size, len1, len2, self.output_dim))

        # Puerta 
        g = K.sigmoid(K.dot(e, self.Wg) + self.bg)
        g = K.reshape(g, (batch_size, len1, len2, self.output_dim))

        # Relevancia
        s = K.dot(g*btp + (1-g)*sln + self.b, self.u)

        return s

    def compute_output_shape(self, input_shape):
        shape1 = input_shape[0]
        shape2 = input_shape[1]
        return (shape1[0], shape1[1], shape2[1], 1)

In [ ]:
def create_model(input_shape, embeddings_dim, embeddings_matrix, vocab_size,
                      max_seq_length, trainable_embeddings, dropout,
                      lstm_hidden_units, attention_channels, pool_size,
                      fc_hidden_units):
   
    X1_input = Input(input_shape, name="input_X1")
    X2_input = Input(input_shape, name="input_X2")

    # Embeddings
    embeddings = Embedding(vocab_size,
                    embeddings_dim,
                    weights=[embeddings_matrix],
                    input_length=input_shape[0],
                    trainable=trainable_embeddings,
                    mask_zero=False)
    X1 = embeddings(X1_input)
    X2 = embeddings(X2_input)
    
    # Recurrencia
    encoder = Bidirectional(LSTM(lstm_hidden_units, return_sequences=True))

    X1_encoded = encoder(X1)
    X2_encoded = encoder(X2)

    # Mecanismo de atención 
    X = GatedRelevanceNetwork(attention_channels, name="grn")([X1_encoded, X2_encoded])
    #X = BatchNormalization()(X)

    # Reducción de dimensionalidad
    X = MaxPooling2D(pool_size=(pool_size, pool_size), strides=(pool_size, pool_size), 
                     padding='valid',
                        data_format="channels_last",
                        name="max_pool")(X)
    X = Flatten()(X)

    # Clasificación
    X = Dense(fc_hidden_units, activation="tanh", name="mlp")(X)
    X = Dropout(dropout)(X)
    X = Dense(2, activation="softmax", name="output")(X)

    model = Model(inputs=[X1_input, X2_input], outputs=X, name="GRN_model")

    return model


dropout = 0.5
trainable_embeddings = False
lstm_hidden_units = 50
attention_channels = 2
pool_size = 3
fc_hidden_units = 128
use_class_weight = False
input_shape = (max_size,)
embedding_dim = 300
vocab_size = total_tokens

model = create_model(input_shape,
                      embedding_dim, embedding_matrix, vocab_size,
                      max_size, trainable_embeddings, dropout,
                      lstm_hidden_units, attention_channels, pool_size,
                      fc_hidden_units)
model.summary()

In [ ]:
from keras import optimizers
optimizer = optimizers.Adam()
model.compile(optimizer=optimizer,
            loss="categorical_crossentropy",
            metrics=["accuracy"])
model.fit([X1, X2], to_categorical(train.label.values),
          epochs=10,
          batch_size=32,
          validation_data=([X_val1, X_val2], to_categorical(test.label.values)),
          callbacks=[PlotLossesKerasTF()])